In [3]:
import numpy as np
import pdb
from PIL import Image
from scipy import ndimage
import os
import unittest
from scipy.ndimage import maximum_filter

In [289]:
png_file = Image.open("C:\\Data\\2.png")
array_image = np.array(png_file)
kernel_3d = np.random.rand(3, 3, 4)
kernel_32d = np.random.rand(3, 3, 32)
png_file_1 = Image.open("C:\\Data\\1.png")
array_image_1 = np.array(png_file_1)
png_file_2 = Image.open("C:\\Data\\7.png")
array_image_2 = np.array(png_file_2)

In [120]:
array_image_1.shape

(343, 345, 4)

In [316]:
test = np.random.rand(294, 292, 32)

In [317]:
filtry = create_kernels(64, 32)

In [98]:
to_stack = []

In [5]:
kernel_3d

array([[[0.97579188, 0.09795808, 0.59583553, 0.5651619 ],
        [0.49750619, 0.06103113, 0.02122378, 0.60717651],
        [0.3420268 , 0.45209318, 0.99694965, 0.68103131]],

       [[0.17337962, 0.29205918, 0.0787763 , 0.63871139],
        [0.38942155, 0.53914352, 0.359275  , 0.9200078 ],
        [0.00591711, 0.61160558, 0.67066265, 0.93274303]],

       [[0.6657244 , 0.73505632, 0.80927753, 0.5256376 ],
        [0.6160557 , 0.79009258, 0.32372186, 0.64066075],
        [0.10862612, 0.6651775 , 0.85617951, 0.33736937]]])

In [87]:
def reshape_dimensions_convolution(input_height, input_width, filter_height, filter_width, stride):
    output_height = int(((input_height - filter_height) / stride) + 1)
    output_width  = int(((input_width - filter_width)  / stride) + 1)
    return (output_height, output_width)

In [55]:
def reshape_dimensions_pooling(input_height, input_width, f, s):
    output_height = int(((input_height - f) / s) + 1)
    output_width = int(((input_width - f) / s) + 1)
    return (output_height, output_width)

In [121]:
def forward_pass(image):
    shape = image.shape
    kernels = create_kernels(32, 4)
    kernels2 =  create_kernels(64, 32)
    to_stack_values = []
    dimensions = reshape_dimensions_convolution(shape[0], shape[1], 3, 3, 1) 
    for i in range(len(kernels)):
        to_stack_values.append(np.squeeze(convolve_one_step(image, kernels[i]).reshape(dimensions[0], dimensions[1], 1)))
    pdb.set_trace()
    stacked_filters = np.stack(to_stack_values, axis = 2)
    first_ReLU = ReLUv2(stacked_filters)
    pool_to_reshape = reshape_dimensions_pooling(first_ReLU.shape[0], first_ReLU.shape[1], 2, 2)
    first_max_pool = np.array(max_pooling(first_ReLU)).reshape(pool_to_reshape[0], pool_to_reshape[1], 32)
    to_stack_values.clear()
    shape2 = first_max_pool.shape
    dimensions2 = reshape_dimensions_convolution(shape2[0], shape2[1], 3, 3, 1)
    for f in range(len(kernels2)):
        to_stack_values.append(np.squeeze(convolve_one_step(first_max_pool, kernels2[f]).reshape(dimensions2[0], dimensions2[1], 1)))
    stacked_filters2 = np.stack(to_stack_values, axis = 2)
    second_ReLU = ReLUv2(stacked_filters2)
    pool_to_reshape2 = reshape_dimensions_pooling(second_ReLU.shape[0], second_ReLU.shape[1], 2, 2)
    second_max_pool = np.array(max_pooling(second_ReLU).reshape(pool_to_reshape[2], pool_to_reshape[2]), 64)
    return second_max_pool
    

In [332]:
shajp = test.shape

In [335]:
to_reshape = reshape_dimensions_convolution(shajp[0], shajp[1], 3, 3, 1)

In [339]:
for f in range(len(filtry)):
    to_stack.append(np.squeeze(convolve_one_step(test, filtry[f]).reshape(to_reshape[0], to_reshape[1], 1)))

In [341]:
alfa = np.stack(to_stack, axis = 2)

In [343]:
alfa.shape

(292, 290, 64)

In [314]:
def create_kernels(number_to_create, depth):
    every_kernel = []
    for f_u in range(number_to_create):
        every_kernel.append(np.random.rand(3, 3, depth))
    return every_kernel

In [310]:
test_image = np.random.rand(10, 10, 4)

In [307]:
def convolve_one_step(input_image, kernel_used):
    intermediate_values = []
    dimensions = input_image.shape
    final_values = []
    for i in range(dimensions[0] - 2):
        for j in range(dimensions[1] - 2):
            for k in range(dimensions[2]):
                slice_of_image = input_image[i : i + 3, j : j + 3, k]
                real_kernel = kernel_used[:, :, k]
                value = np.vdot(slice_of_image, real_kernel)
                intermediate_values.append(value)
                #pdb.set_trace()
                if len(intermediate_values) == dimensions[2]:
                    final_values.append(sum_dimension(intermediate_values))
                    intermediate_values.clear()
    return np.array(final_values)

In [20]:
def sum_dimension(list_of_three):
    return sum(list_of_three)

In [21]:
image_after_conv_2 = np.array(convolve_one_step(array_image, kernel_3d)).reshape(dimensions[0], dimensions[1], 1)

NameError: name 'dimensions' is not defined

In [ ]:
image_after_conv_2.shape

In [ ]:
image_after_conv_1 = np.array(convolve_one_step(array_image_1, kernel_3d)).reshape(341, 343, 1)

In [275]:
a = np.array(convolve_one_step(test_image, kernel_3d)).reshape(8, 8, 1)

In [276]:
b = np.array(convolve_one_step(test_image, kernel_3d_2)).reshape(8, 8, 1)

In [277]:
c = np.array(convolve_one_step(test_image, kernel_3d_3)).reshape(8, 8, 1)

In [278]:
a.shape

(8, 8, 1)

In [279]:
a = a.squeeze()

In [280]:
b = b.squeeze()

In [281]:
c = c.squeeze()

In [283]:
a

array([[ 9.54790164, 10.6565483 ,  9.39649574,  9.64609363, 10.13447533,
        11.03573752, 11.85757551, 10.67386543],
       [ 9.12903275,  9.82200007,  9.27298965,  9.69934024, 11.32673874,
        11.3287305 , 11.22928668,  9.28639457],
       [ 8.60544452,  9.32410902, 10.75884813,  9.71445442,  9.5567037 ,
        10.57367801, 10.02161708, 10.3587956 ],
       [ 9.5034332 ,  9.34867802,  9.68787524,  9.83468329,  8.90299788,
         8.38941811,  9.89182205, 10.22037535],
       [ 8.47879535,  9.60069878, 10.7595178 ,  9.58167921,  9.03443968,
         9.0630678 , 10.51611851, 11.1369592 ],
       [ 9.30176348,  9.44569243, 10.70986896,  9.79774711,  9.27245723,
        10.50470211, 11.12966361, 11.04197597],
       [ 9.11033151, 10.44602876,  9.71948831, 10.60129959,  9.31053942,
        10.1052088 , 11.02665671, 10.95409363],
       [ 8.78549919,  8.96082636,  7.77460272,  7.08828344,  7.38471261,
         9.94769685, 10.93531029, 11.14555735]])

In [284]:
b

array([[ 9.44924823,  9.89445793,  8.29965551,  9.69743835, 10.98087816,
        11.7184895 , 11.25942447, 10.01320655],
       [ 9.20639454, 10.69517855,  9.1236393 ,  9.16649243,  9.14245804,
        11.9406294 , 10.28018141, 10.12435873],
       [ 8.76281344,  9.35448148, 10.17868114,  9.1634896 ,  9.03586633,
         9.69253807, 10.05097906,  9.73144927],
       [ 8.55459851,  9.56132553,  9.95898043,  9.10153536,  8.32361792,
         8.17239369,  9.840852  , 10.70035263],
       [ 8.24774563,  8.06783479, 10.8915276 ,  9.89632226,  9.19344604,
         8.0989777 , 10.14991029, 10.89533519],
       [ 9.27240544,  9.25579034,  9.51313396, 10.30546283,  8.34317553,
        10.08760486,  9.87835541, 11.00567937],
       [ 9.26901676, 10.36363174, 10.11051291,  9.28225436, 10.16545403,
         9.82278364, 10.77662643, 10.51914221],
       [ 7.98286722,  8.47089819,  8.40780529,  7.75603283,  7.87150752,
        10.497689  , 10.45495576, 10.40175168]])

In [285]:
c

array([[ 8.89794567,  9.61986927,  9.18505854,  8.91740733, 10.73067685,
        10.61628627, 10.96966588,  9.43857622],
       [ 8.5750684 ,  9.15150425,  9.43470843,  9.04332171,  9.581106  ,
         9.78461478,  9.14772544,  9.2736566 ],
       [ 8.81982703,  8.17921697, 10.04416932,  9.03973521,  8.91279471,
         8.91489242,  8.733983  ,  9.65893087],
       [ 8.16943651,  8.44038798,  8.90873136,  9.01932826,  7.90989837,
         8.36341163,  8.16332222, 10.01002449],
       [ 8.28337983,  7.87683988,  9.6495377 ,  8.78439104,  8.07320467,
         9.36065831,  9.22800095,  9.90966799],
       [ 9.57276987,  8.90576106,  9.79055781,  9.32551585,  9.35601951,
         9.32483334,  9.66650855, 10.62266264],
       [ 9.44276301,  9.25114494,  9.63752208,  8.71254021,  8.62808516,
        10.01531609,  9.82879683, 10.39414203],
       [ 7.57336225,  7.29284313,  8.02370963,  6.99035861,  6.82089897,
         9.69505497, 10.01872633,  9.8090814 ]])

In [286]:
dupa_tymona = np.stack([a, b, c], axis = 2)

In [287]:
dupa_tymona.shape

(8, 8, 3)

In [38]:
def max_pooling(image_array):
    pooling_values = []
    for m in range(0, image_array.shape[0], 2):
        for n in range(0, image_array.shape[1], 2):
            for o in range(image_array.shape[2]):
                if n + 2 > image_array.shape[1]:
                    slice_of_pooling = image_array[m : m + 2, n - 1: n + 1 , o]
                else:
                    slice_of_pooling = image_array[m : m + 2, n : n + 2 , o]
                pooling_values.append(slice_of_pooling.max())
                #pdb.set_trace()
    return pooling_values

In [ ]:
filtr = test_image[0 : 2, 0 : 2, 0]

In [ ]:
after_maxpool = np.array(max_pooling(test_image)).reshape(5, 5, 4)

In [ ]:
after_maxpool.shape

In [28]:
def ReLUv1(x):
    after_relu = []
    for r in x:
        if r > 0:
            after_relu.append(r)
        else:
            after_relu.append(0)
    return after_relu

In [29]:
def ReLUv2(x):
    return np.maximum(0, x)

In [30]:
last_pool = np.random.rand(2, 2, 1)

In [31]:
def fully_connected(after_last_max_pool, number_of_nodes):
    almost_values = []
    nodes_values = []
    flatten_array = after_last_max_pool.reshape(4)
    weights = np.random.rand(len(flatten_array), number_of_nodes)
    for x in range(weights.shape[1]):
        for y in range(weights.shape[0]):
            if y == weights.shape[0] - 1:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y])
                nodes_values.append(sum(almost_values))
                almost_values.clear()
            else:
                almost_values.append(weights[y : y + 1, x] * flatten_array[y])
    return nodes_values

In [32]:
fully_connected(last_pool, 4)

[array([0.37035173]),
 array([0.14957847]),
 array([0.2416212]),
 array([0.28055913])]

In [33]:
lista = []
for i in range(10):
    kernel = np.random.rand(3, 3, 4)
    lista.append(kernel)